In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot"

#### install & import

In [ ]:
!pip install --upgrade pip

!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece # sentencepiece 설치하고 무조건 런타임 재시작해야함
!pip install transformers
!pip install torch
!pip install kobert_transformers

# 토크나이저 관련 파일 install
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
# 필요 라이브러리 import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer

#transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


#### 감정분류  
https://hoit1302.tistory.com/159#4.%EC%8B%A4%EC%8A%B5%EC%84%A4%EB%AA%85,%EC%BD%94%EB%93%9C

##### import

In [ ]:
# 필요 라이브러리 import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

from sklearn.model_selection import train_test_split
# from kobert_transformers import get_tokenizer
# from kobert_tokenizer import KoBERTTokenizer

#transformers
# from transformers import BertModel
from transformers import BertTokenizer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용 시
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

##### dataset 불러오기

In [ ]:
# 데이터셋 불러오기
data_1 = pd.read_csv(path+'/dataset/emotion_classification/4차년도.csv', encoding='cp949')
print(data_1.shape) # (14606, 15)

data_2 = pd.read_csv(path+'/dataset/emotion_classification/5차년도.csv', encoding='cp949')
print(data_2.shape) # (10011, 15)

data_3 = pd.read_csv(path+'/dataset/emotion_classification/5차년도_2차.csv', encoding='cp949')
print(data_3.shape) # (19374, 15)

print(data_1['상황'].unique()) # 'anger', 'sad', 'fear', 'disgust', 'neutral'
print(data_2['상황'].unique()) # 'disgust' 'sad' 'anger' 'fear' 'neutral'
print(data_3['상황'].unique()) # 'happiness' 'neutral' 'sadness' 'angry' 'surprise' 'disgust' 'fear'

(14606, 15)
(10011, 15)
(19374, 15)
['anger' 'sad' 'fear' 'disgust' 'neutral']
['disgust' 'sad' 'anger' 'fear' 'neutral']
['happiness' 'neutral' 'sadness' 'angry' 'surprise' 'disgust' 'fear']


In [ ]:
# data_1 ~ data_3 concat
data = pd.concat([data_1, data_2, data_3], ignore_index=True)
print(data.shape) # (43991, 15)
data.head(3)

(43991, 15)


,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,anger,Angry,1,Angry,1,Neutral,0,Angry,1,Angry,1,27,male


In [ ]:
data.loc[(data['상황'] == "fear"), '상황'] = 0      #공포 => 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
data.loc[(data['상황'] == "anger"), '상황'] = 2     #분노 => 2
data.loc[(data['상황'] == "angry"), '상황'] = 2     # ''
data.loc[(data['상황'] == "sad"), '상황'] = 3       #슬픔 => 3
data.loc[(data['상황'] == "sadness"), '상황'] = 3   # ''
data.loc[(data['상황'] == "neutral"), '상황'] = 4   #중립 => 4
data.loc[(data['상황'] == "happiness"), '상황'] = 5 #행복 => 5
data.loc[(data['상황'] == "disgust"), '상황'] = 6   #혐오 => 6

In [ ]:
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

# data_list
# [['어, 청소 니가 대신 해 줘!', '2'],
#  ['둘 다 청소 하기 싫어. 귀찮아.', '2'],
#  ['둘 다 하기 싫어서 화내.', '2'],
#  ['그럼 방세는 어떡해.', '2'],
#  ['권태긴줄 알았는데 다른 사람이 생겼나보더라고.', '3'],
#                     ...                           ]

In [ ]:
data_list[:5]

[['어, 청소 니가 대신 해 줘!', '2'],
 ['둘 다 청소 하기 싫어. 귀찮아.', '2'],
 ['둘 다 하기 싫어서 화내.', '2'],
 ['그럼 방세는 어떡해.', '2'],
 ['권태긴줄 알았는데 다른 사람이 생겼나보더라고.', '3']]

##### dataset tokenize

In [ ]:
# BERT 모델의 입력으로 넣기 위한 데이터셋 토큰화
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

##### Setting parameters

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

##### train_test_split & dataloader

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


##### KoBERT model

In [ ]:
# KoBERT
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


##### train

In [ ]:
# train
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-26-d1b3de9d5888>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/550 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9622658491134644 train acc 0.125
epoch 1 batch id 201 loss 0.7000296711921692 train acc 0.45133706467661694
epoch 1 batch id 401 loss 0.6773717403411865 train acc 0.6433915211970075
epoch 1 train acc 0.7052313311688312


<ipython-input-26-d1b3de9d5888>:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/138 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9293259116409537


  0%|          | 0/550 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.028597015887498856 train acc 1.0
epoch 5 batch id 201 loss 0.09657548367977142 train acc 0.9738028606965174
epoch 5 batch id 401 loss 0.1858462691307068 train acc 0.975218204488778
epoch 5 train acc 0.9758198051948053


  0%|          | 0/138 [00:00<?, ?it/s]

epoch 5 test acc 0.9323756720430108


In [ ]:
# model save
save_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot/model/"

torch.save(model, save_path + 'emotion_model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), save_path + 'emotion_state_model.pt')  # 모델 객체의 state_dict 저장

##### predict test

In [ ]:
# model load
load_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot/model/"

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load(load_path + "emotion_model.pt", map_location=device)
model

In [ ]:
# 예측 테스트

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요(종료 0) : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요(종료 0) : 나 오늘 엄청 짜증나
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요(종료 0) : 진짜 행복한 하루였어
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요(종료 0) : 0


#### KoBERT Chatbot
https://hoit1302.tistory.com/162#[2]kobert%EA%B8%B0%EB%B0%98%EC%8B%AC%EB%A6%AC%EC%BC%80%EC%96%B4%EC%B1%97%EB%B4%87  
https://github.com/hoit1302/kobert-wellness-chatbot/blob/main/model/dataloader.py

##### configuration

In [ ]:
# 설정값 정의
import logging

from transformers import BertConfig

logger = logging.getLogger(__name__)

kobert_config = {
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'gelu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 768,
    'initializer_range': 0.02,
    'intermediate_size': 3072,
    'max_position_embeddings': 512,
    'num_attention_heads': 12,
    'num_hidden_layers': 12,
    'type_vocab_size': 2,
    'vocab_size': 8002
}

def get_kobert_config():
    return BertConfig.from_dict(kobert_config)

##### classifier

In [ ]:
# 감정 분류기 : kobert 모델을 사용해 먼저 어떤 상황인지 분류

import torch
import torch.nn as nn
from kobert_transformers import get_kobert_model
from torch.nn import CrossEntropyLoss, MSELoss
from transformers import BertModel
# from tranaformers import BertPreTrainedModel

class KoBERTforSequenceClassfication(BertModel):
    def __init__(self,
                 num_labels=359,
                 hidden_size=768,
                 hidden_dropout_prob=0.1,
                 ):
        super().__init__(get_kobert_config())

        self.num_labels = num_labels
        self.kobert = get_kobert_model()
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.classifier = nn.Linear(hidden_size, num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
    ):
        outputs = self.kobert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

##### dataloader

In [ ]:
# dataloader
# csv 데이터를 pytorch 모델로 학습시키기 위함
import torch
# from kobert_transformers import get_tokenizer
from transformers import BertTokenizer
from torch.utils.data import Dataset

root_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot/dataset/wellness_script"

class WellnessTextClassificationDataset(Dataset):
    def __init__(self,
                 file_path=root_path+"/wellness_dialog_for_text_classification_all.txt",
                 num_label=359,
                 device='cpu',
                 max_seq_len=512,  # KoBERT max_length
                 tokenizer=None
                 ):
        self.file_path = file_path
        self.device = device
        self.data = []
        # self.tokenizer = tokenizer if tokenizer is not None else get_tokenizer()
        self.tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

        file = open(self.file_path, 'r', encoding='utf-8')

        while True:
            line = file.readline()
            if not line:
                break
            datas = line.split("    ")
            index_of_words = self.tokenizer.encode(datas[0])
            token_type_ids = [0] * len(index_of_words)
            attention_mask = [1] * len(index_of_words)

            # Padding Length
            padding_length = max_seq_len - len(index_of_words)

            # Zero Padding
            index_of_words += [0] * padding_length
            token_type_ids += [0] * padding_length
            attention_mask += [0] * padding_length

            # Label
            label = int(datas[1][:-1])

            data = {
                'input_ids': torch.tensor(index_of_words).to(self.device),
                'token_type_ids': torch.tensor(token_type_ids).to(self.device),
                'attention_mask': torch.tensor(attention_mask).to(self.device),
                'labels': torch.tensor(label).to(self.device)
            }

            self.data.append(data)

        file.close()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        return item


if __name__ == "__main__":
    dataset = WellnessTextClassificationDataset()
    print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

##### train

In [ ]:
# train
import gc
import os

import numpy as np
import torch
from torch.utils.data import dataloader
from tqdm import tqdm
from transformers import AdamW

# from model.classifier import KoBERTforSequenceClassfication
# from model.dataloader import WellnessTextClassificationDataset

def train(device, epoch, model, optimizer, train_loader, save_step, save_ckpt_path, train_step=0):
    losses = []
    train_start_index = train_step + 1 if train_step != 0 else 0
    total_train_step = len(train_loader)
    model.train()

    with tqdm(total=total_train_step, desc=f"Train({epoch})") as pbar:
        pbar.update(train_step)
        for i, data in enumerate(train_loader, train_start_index):

            optimizer.zero_grad()
            outputs = model(**data)

            loss = outputs[0]

            losses.append(loss.item())

            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

            if i >= total_train_step or i % save_step == 0:
                torch.save({
                    'epoch': epoch,  # 현재 학습 epoch
                    'model_state_dict': model.state_dict(),  # 모델 저장
                    'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 저장
                    'loss': loss.item(),  # Loss 저장
                    'train_step': i,  # 현재 진행한 학습
                    'total_train_step': len(train_loader)  # 현재 epoch에 학습 할 총 train step
                }, save_ckpt_path)

    return np.mean(losses)


if __name__ == '__main__':
    gc.collect()
    torch.cuda.empty_cache()

    root_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot"
    data_path = f"{root_path}/dataset/wellness_script/wellness_dialog_for_text_classification_all.txt"
    checkpoint_path = f"{root_path}/checkpoint"
    save_ckpt_path = f"{checkpoint_path}/kobert-wellness-text-classification.pth"

    n_epoch = 50  # Num of Epoch (원래는 60 에폭)
    batch_size = 4   # 배치 사이즈 : Colab이 돌아가지 않아 4로 했으며, 증가시켜도 무방
    ctx = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(ctx)
    save_step = 100  # 학습 저장 주기
    learning_rate = 5e-6  # Learning Rate

    # WellnessTextClassificationDataset Data Loader
    dataset = WellnessTextClassificationDataset(file_path=data_path, device=device)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = KoBERTforSequenceClassfication()
    model.to(device)

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

    pre_epoch, pre_loss, train_step = 0, 0, 0
    if os.path.isfile(save_ckpt_path):
        checkpoint = torch.load(save_ckpt_path, map_location=device)
        pre_epoch = checkpoint['epoch']
        train_step = checkpoint['train_step']
        total_train_step = checkpoint['total_train_step']

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")

    losses = []
    offset = pre_epoch
    for step in range(n_epoch):
        epoch = step + offset
        loss = train(device, epoch, model, optimizer, train_loader, save_step, save_ckpt_path, train_step)
        losses.append(loss)

In [ ]:
save_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot/model/"

torch.save(model, save_path + 'kobert.pt')  # 전체 모델 저장
torch.save(model.state_dict(), save_path + 'kobert_state_dict.pt')  # 모델 객체의 state_dict 저장

##### test

In [ ]:
# model load
load_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot/model/"

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load(load_path + "kobert-wellness-text-classification.pth", map_location=device)
model

In [ ]:
# test
import torch
import torch.nn as nn
import random

from kobert_transformers import get_tokenizer
from transformers import BertTokenizer

def load_wellness_answer():
    root_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot"
    category_path = f"{root_path}/dataset/wellness_script/wellness_dialog_category.txt"
    answer_path = f"{root_path}/dataset/wellness_script/wellness_dialog_answer.txt"

    c_f = open(category_path, 'r')
    a_f = open(answer_path, 'r')

    category_lines = c_f.readlines()
    answer_lines = a_f.readlines()

    category = {}
    answer = {}
    for line_num, line_data in enumerate(category_lines):
        data = line_data.split('    ')
        category[data[1][:-1]] = data[0]

    for line_num, line_data in enumerate(answer_lines):
        data = line_data.split('    ')
        keys = answer.keys()
        if (data[0] in keys):
            answer[data[0]] += [data[1][:-1]]
        else:
            answer[data[0]] = [data[1][:-1]]

    return category, answer


def kobert_input(tokenizer, str, device=None, max_seq_len=512):
    index_of_words = tokenizer.encode(str)
    token_type_ids = [0] * len(index_of_words)
    attention_mask = [1] * len(index_of_words)

    # Padding Length
    padding_length = max_seq_len - len(index_of_words)

    # Zero Padding
    index_of_words += [0] * padding_length
    token_type_ids += [0] * padding_length
    attention_mask += [0] * padding_length

    data = {
        'input_ids': torch.tensor([index_of_words]).to(device),
        'token_type_ids': torch.tensor([token_type_ids]).to(device),
        'attention_mask': torch.tensor([attention_mask]).to(device),
    }
    return data


if __name__ == "__main__":
    root_path = "/content/drive/MyDrive/02.Project/Bit_PoscoDX/chatbot"
    checkpoint_path = f"{root_path}/model"
    # model_path = f"{root_path}/model"
    save_ckpt_path = f"{checkpoint_path}/kobert-wellness-text-classification.pth"

    # 답변과 카테고리 불러오기
    category, answer = load_wellness_answer()

    ctx = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(ctx)

    # 저장한 Checkpoint 불러오기
    checkpoint = torch.load(save_ckpt_path, map_location=device)

    model = KoBERTforSequenceClassfication()
    model.load_state_dict(checkpoint['model_state_dict'])

    model.to(ctx)
    model.eval()

    # tokenizer = get_tokenizer()
    tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

    while 1:
        sent = input('\nQuestion: ')  # '요즘 기분이 우울한 느낌이에요'
        data = kobert_input(tokenizer, sent, device, 512)

        if '종료' in sent:
            break

        output = model(**data)

        logit = output[0]
        softmax_logit = torch.softmax(logit, dim=-1)
        softmax_logit = softmax_logit.squeeze()

        max_index = torch.argmax(softmax_logit).item()
        max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

        answer_list = answer[category[str(max_index)]]
        answer_len = len(answer_list) - 1
        answer_index = random.randint(0, answer_len)
        print(f'Answer: {answer_list[answer_index]}, index: {max_index}, softmax_value: {max_index_value}')
        print('-' * 50)


Question: 특별한 이유가 없는데 그냥 불안하고 눈물이 나와
Answer: 당신은 혼자가 아니에요. 제가 옆에 있다는 것만 기억해주세요., index: 68, softmax_value: 0.06762945652008057
--------------------------------------------------

Question: 이 세상에서 완전히 사라지고 싶어
Answer: 그런 일이 있으셨군요. 조금씩 나아질 거라고 믿어요., index: 84, softmax_value: 0.1267862617969513
--------------------------------------------------

Question: 남들이 나를 어떻게 생각할지 신경쓰게 돼
Answer: 네, 그렇군요. 듣고 있으니 계속 말씀하세요., index: 90, softmax_value: 0.5786895751953125
--------------------------------------------------

Question: 내일 놀이공원 갈건데 사람 별로 없었으면 좋겠다
Answer: 작은 것부터 기억하려고 노력해보는 건 어떨까요?, index: 271, softmax_value: 0.11624133586883545
--------------------------------------------------

Question: 종료
